In [34]:
import cv2
import json
import os
import numpy as np

# Load the annotations JSON file
json_path = r'video1.json'
with open(json_path, 'r') as f:
    annotations = json.load(f)

# Output directories for images and annotations
output_img_dir = 'output_images'
output_txt_dir = 'output_annotations'
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_txt_dir, exist_ok=True)

# Define the FPS of the video and Label Studio
video_fps = 59.19  # Actual video FPS
label_studio_fps = 25.04  # Label Studio FPS
scaling_factor = video_fps / label_studio_fps  # Scaling factor

# Define a mapping from label names to class IDs
label_to_class_id = {
    "1 star": 0, "2 stars": 1, "3 stars": 2, "4 stars": 3, "5 stars": 4, "6 stars": 5,
    "7 stars": 6, "8 stars": 7, "Location": 8, "big monster": 9, "monster": 10,
    "monster hp": 11, "monster name": 12, "player": 13, "player hp": 14, "player name": 15,
    "quest": 16, "small monster": 17, "time": 18, "ultimate not ready": 19, "ultimate ready": 20
}

# Function to check if a bounding box should be drawn in the current frame
def get_boxes_for_frame(sequence, frame_number):
    for i in range(len(sequence) - 1):
        start_box = sequence[i]
        end_box = sequence[i + 1]

        # Scale Label Studio frames to match video FPS
        start_frame = int(start_box['frame'] * scaling_factor)
        end_frame = int(end_box['frame'] * scaling_factor)
        
        # Check if the current frame falls between the start_frame and end_frame
        if start_frame <= frame_number < end_frame:
            if start_box['enabled']:
                return {
                    'x': start_box['x'],
                    'y': start_box['y'],
                    'width': start_box['width'],
                    'height': start_box['height'],
                    'labels': start_box.get('labels', [])
                }
    return None

# Function to write YOLO format annotations to a text file
def write_yolo_annotations(txt_file_path, boxes, frame_width, frame_height):
    with open(txt_file_path, 'w') as f:
        for box in boxes:
            if box:
                # Get normalized YOLO format values
                x_center = (box['x'] + box['width'] / 2) / 100  # x_center in percent
                y_center = (box['y'] + box['height'] / 2) / 100  # y_center in percent
                width = box['width'] / 100  # width in percent
                height = box['height'] / 100  # height in percent

                # Get the class ID for the label
                if 'labels' in box and box['labels']:
                    class_id = label_to_class_id.get(box['labels'][0], 0)
                else:
                    class_id = 0  # Default to 0 if no label found

                # Write to the annotation file in YOLO format
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process each video in the annotations
for annotation in annotations:
    video_path = annotation['video']  # Update this to your local path if needed
    video_filename = os.path.basename(video_path)

    # Load video
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Process frame by frame
    frame_number = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Collect all boxes for this frame from all classes
        frame_boxes = []
        for box_annotation in annotation['box']:
            sequence = box_annotation['sequence']
            box_for_frame = get_boxes_for_frame(sequence, frame_number)
            if box_for_frame:
                # Attach labels from the box annotation
                box_for_frame['labels'] = box_annotation.get('labels', [])
                frame_boxes.append(box_for_frame)

        # Save frame as image
        image_filename = f"frame_{frame_number}.jpg"
        image_path = os.path.join(output_img_dir, image_filename)
        cv2.imwrite(image_path, frame)

        # Write the annotations for the current frame
        txt_filename = f"frame_{frame_number}.txt"
        txt_path = os.path.join(output_txt_dir, txt_filename)
        write_yolo_annotations(txt_path, frame_boxes, width, height)

        frame_number += 1
        if frame_number >= total_frames:
            break

    cap.release()

print("Processing complete! Frames and annotations have been saved.")


Processing complete! Frames and annotations have been saved.


In [26]:
import cv2

# Function to get the FPS and total frames of the video
def get_video_info(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file")
        return None, None
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total number of frames
    cap.release()
    return fps, total_frames

# Function to calculate the limited number of frames based on a target FPS
def calculate_limited_frames(original_fps, total_frames, target_fps):
    if target_fps > original_fps:
        print("Target FPS is higher than the original FPS, no need to limit.")
        return total_frames
    limited_frames = int((target_fps / original_fps) * total_frames)
    return limited_frames

# Example usage
video_path = r'C:\Users\lewka\deep learning\MonsterHNow\video\video_from\dd36f5e0-8.mp4'  # Replace with your actual video file path
target_fps = 25.07  # Set your desired FPS

# Get the original video information
fps, total_frames = get_video_info(video_path)
if fps is not None and total_frames is not None:
    print(f"Original FPS of the video: {fps}")
    print(f"Total number of frames in the video: {total_frames}")

    # Calculate the limited number of frames
    limited_frames = calculate_limited_frames(fps, total_frames, target_fps)
    print(f"Total number of frames at {target_fps} FPS: {limited_frames}")


Original FPS of the video: 59.18777292576419
Total number of frames in the video: 1506
Total number of frames at 25.07 FPS: 637


In [36]:
import cv2
import os

# Directories containing the extracted images and YOLO annotations
image_dir = 'output_images'
annotation_dir = 'output_annotations'

# Define the output video path
output_video_path = 'recreated_video.mp4'

# Get sorted list of images by frame number to ensure sequential order
image_list = sorted(os.listdir(image_dir), key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))

# Check if the directory is not empty
if not image_list:
    raise Exception("No images found in the directory")

# Load the first image to get video properties (width, height)
first_image = cv2.imread(os.path.join(image_dir, image_list[0]))
height, width, _ = first_image.shape

# Create VideoWriter to save the recreated video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 25, (width, height))  # 25 FPS is a placeholder, it won't impact the sequence

# Function to load YOLO annotations from a text file
def load_yolo_annotations(txt_file_path):
    boxes = []
    with open(txt_file_path, 'r') as f:
        for line in f.readlines():
            box_data = line.strip().split()
            class_id = int(box_data[0])
            x_center = float(box_data[1])
            y_center = float(box_data[2])
            width = float(box_data[3])
            height = float(box_data[4])

            boxes.append((class_id, x_center, y_center, width, height))
    return boxes

# Function to draw bounding boxes on a frame based on YOLO annotations
def draw_yolo_boxes(frame, boxes, frame_width, frame_height):
    for box in boxes:
        class_id, x_center, y_center, width, height = box
        
        # Convert YOLO format (normalized) back to pixel values
        x_center_px = int(x_center * frame_width)
        y_center_px = int(y_center * frame_height)
        box_width_px = int(width * frame_width)
        box_height_px = int(height * frame_height)
        
        # Calculate the top-left corner of the bounding box
        x_min = x_center_px - box_width_px // 2
        y_min = y_center_px - box_height_px // 2
        
        # Draw rectangle around the object
        cv2.rectangle(frame, (x_min, y_min), (x_min + box_width_px, y_min + box_height_px), (0, 255, 0), 2)
        
        # Draw the class ID on top of the box
        label = f"Class: {class_id}"
        cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

# Process each frame and corresponding annotation file
for image_file in image_list:
    # Load the image
    image_path = os.path.join(image_dir, image_file)
    frame = cv2.imread(image_path)

    # Get the corresponding annotation file (.txt) based on the image filename
    frame_number = os.path.splitext(image_file)[0]  # Extract "frame_x" from the filename
    annotation_file = f"{frame_number}.txt"
    annotation_path = os.path.join(annotation_dir, annotation_file)

    # If the annotation file exists, load the bounding boxes
    if os.path.exists(annotation_path):
        boxes = load_yolo_annotations(annotation_path)
        # Draw the bounding boxes on the frame
        frame = draw_yolo_boxes(frame, boxes, width, height)

    # Write the frame to the video
    out.write(frame)

# Release the VideoWriter
out.release()

print(f"Recreated video saved at {output_video_path}")


Recreated video saved at recreated_video.mp4
